Load the python packages

In [ ]:
import pandas as pd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math


%matplotlib inline

load three datasets

In [ ]:
aw_custs = pd.read_csv('AdvWorksCusts.csv')
aw_aveMonth = pd.read_csv('AW_AveMonthSpend.csv')
aw_bike=pd.read_csv('AW_BikeBuyer.csv')

In [ ]:
print(aw_custs.dtypes)
aw_custs.head()
aw_custs.describe()

check the number of NAN in each column

In [ ]:
for col in aw_custs.columns:
    if aw_custs[col].dtype == object:
        count = 0
        count = [count + 1 for x in aw_custs[col] if pd.notna(x)]
        print(col + ' ' + str(sum(count)))

In [ ]:
drop the colomn which has too many NAN

In [ ]:
aw_custs = aw_custs.drop(['Title','Suffix','AddressLine2'], axis=1)

combine first name and middle name

In [ ]:
def combineName():
    result=[]
    for f, m in zip(aw_custs['FirstName'], aw_custs['MiddleName']):
        if pd.notna(m):
            result.append(f+'.'+m)
        else:
            result.append(f)
    return result
    
aw_custs['FirstName'] = combineName()

In [ ]:
aw_custs = aw_custs.drop(['MiddleName'],axis=1)

In [ ]:
aw_custs.CustomerID.unique().shape
aw_custs.shape
aw_custs.dtypes

In [ ]:
aw_aveMonth.CustomerID.unique().shape
aw_aveMonth.shape
aw_aveMonth.dtypes

In [ ]:
print(aw_bike.CustomerID.unique().shape)
aw_bike.shape

remove rows of duplicate customer id, since id is unique. keep the latest one

In [ ]:
aw_custs.drop_duplicates(subset='CustomerID', keep='last', inplace=True)

In [ ]:
aw_aveMonth.drop_duplicates(subset='CustomerID', keep='last', inplace=True)

In [ ]:
aw_bike.drop_duplicates(subset='CustomerID', keep='last', inplace=True)

In [ ]:
for col in aw_aveMonth.columns:
    if aw_aveMonth[col].dtype == np.int64:
        count = 0
        count = [count + 1 for x in aw_aveMonth[col] if x<0]
        print(col + ' ' + str(sum(count)))

In [ ]:
for col in aw_bike.columns:
    if aw_bike[col].dtype == np.int64:
        count = 0
        count = [count + 1 for x in aw_bike[col] if x>1]
        print(col + ' ' + str(sum(count)))

colomns will be used as features

In [ ]:
def count_unique(aw_custs,cols):
    for col in cols:
        print('\n' + 'For column ' + col)
        print(aw_custs[col].value_counts())
cols=[*aw_custs.columns[6:7],*aw_custs.columns[10:]]
cols
#count_unique(aw_custs,cols)

visualize the features

In [ ]:
def plot_bars(aw_custs,cols):
    for col in cols:
        fig=plt.figure(figsize=(6,6))
        ax=fig.gca()
        counts=aw_custs[col].value_counts()
        counts.plot.bar(ax=ax,color='blue')
        plt.show()
plot_bars(aw_custs,cols)

create dummy variable for those categorical features

In [ ]:
features=aw_custs['CountryRegionName']
enc=preprocessing.LabelEncoder()
enc.fit(features)
features=enc.transform(features)
ohe=preprocessing.OneHotEncoder()
encoded=ohe.fit(features.reshape(-1,1))
features = encoded.transform(features.reshape(-1,1)).toarray()
print(features.shape)

In [ ]:
#features=np.array(aw_custs['HomeOwnerFlag'])
#features=features.reshape(-1,1)
features=np.array(aw_custs['YearlyIncome'])
features=features.reshape(-1,1)
scaler = preprocessing.StandardScaler().fit(features)
features=scaler.transform(features)
print(features)

In [ ]:
def encode_string(feature):
    enc=preprocessing.LabelEncoder()
    enc.fit(feature)
    enc_feature=enc.transform(feature)
    ohe=preprocessing.OneHotEncoder()
    encoded=ohe.fit(enc_feature.reshape(-1,1))
    return encoded.transform(enc_feature.reshape(-1,1)).toarray()
columns=['Education',
 'Occupation',
 'Gender',
 'MaritalStatus',]
for col in columns:
    temp=encode_string(aw_custs[col])
    features=np.concatenate([features,temp],axis=1)
print(features.shape)

In [ ]:
features=np.concatenate([features,np.array(aw_custs[['HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']])],axis=1)
features.shape

create regression model

In [ ]:
nr.seed(123)
labels=np.array(aw_aveMonth['AveMonthSpend'])
index=range(features.shape[0])
index=ms.train_test_split(index,test_size=3000)
x_train=features[index[0],:]
y_train=np.ravel(labels[index[0]])
x_test=features[index[1],:]
y_test=np.ravel(labels[index[1]])

In [ ]:
scaler = preprocessing.StandardScaler().fit(x_train[:,-1:])
x_train[:,-1:]=scaler.transform(x_train[:,-1:])
x_test[:,-1:]=scaler.transform(x_test[:,-1:])
print(x_train.shape)
x_train[:5,:]

In [ ]:
lin_mod=linear_model.LinearRegression(fit_intercept=False)
lin_mod.fit(x_train,y_train)

In [ ]:
print(lin_mod.intercept_)
print(lin_mod.coef_)

test the model

In [ ]:
lin_mod.score(x_test,y_test)

In [ ]:
predict_x=pd.read_csv('AW_test.csv')

In [ ]:
new_features=predict_x['CountryRegionName']
enc=preprocessing.LabelEncoder()
enc.fit(new_features)
new_features=enc.transform(new_features)
ohe=preprocessing.OneHotEncoder()
encoded=ohe.fit(new_features.reshape(-1,1))
new_features = encoded.transform(new_features.reshape(-1,1)).toarray()

In [ ]:
def encode_string(feature):
    enc=preprocessing.LabelEncoder()
    enc.fit(feature)
    enc_feature=enc.transform(feature)
    ohe=preprocessing.OneHotEncoder()
    encoded=ohe.fit(enc_feature.reshape(-1,1))
    return encoded.transform(enc_feature.reshape(-1,1)).toarray()
columns=['Education',
 'Occupation',
 'Gender',
 'MaritalStatus',]
for col in columns:
    temp=encode_string(predict_x[col])
    new_features=np.concatenate([new_features,temp],axis=1)
print(new_features.shape)

In [ ]:
new_features=np.concatenate([new_features,np.array(predict_x[['HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']])],axis=1)
new_features.shape

In [ ]:
scaler = preprocessing.StandardScaler().fit(new_features[:,-1:])
new_features[:,-1:]=scaler.transform(new_features[:,-1:])
print(new_features.shape)
new_features

In [ ]:
predict_y=lin_mod.predict(new_features)

In [ ]:
prediction=pd.DataFrame(predict_y,columns=['prediction']).to_csv('prediction.csv')